In [1]:
from torchvision import models, transforms
from torchsummary import summary
import torch
import numpy as np
import cv2
import PIL
import matplotlib.pyplot as plt

import sys
import os

current_dir = os.getcwd()
sys.path.append(os.path.dirname(current_dir))

from lib.src import *
from lib.utils import *

from pytorch_grad_cam import GradCAM, HiResCAM, AblationCAM, ScoreCAM, GradCAMPlusPlus, XGradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

2024-12-15 15:41:11.683932: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 15:41:11.694790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734273671.708788  330540 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734273671.712931  330540 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 15:41:11.727528: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import random
import os
from tqdm import tqdm


MODEL = resnet50(pretrained=True)
TARGET = MODEL.layer4[-1].conv3

before = []
after = []

PERCENTAGE = 0.01
CLASSES = 200
IMG_PER_CLASS = 5
IN_IMG = 224

VISUALIZE = False

CAM_FUNCTION = get_cam

HEATMAP = 0.5


for dir_name in tqdm(random.sample(os.listdir('imagenet-mini/train'), CLASSES)):
    for file_name in random.sample(os.listdir(f'imagenet-mini/train/{dir_name}'), IMG_PER_CLASS):

        before_img = get_image_torch(f'imagenet-mini/train/{dir_name}/{file_name}', IN_IMG, IN_IMG)
        id, hash, name = DICT[dir_name]

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        bef = out[0, id].item()

        before.append(out[0, id].item())

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)
        argmax = torch.argmax(out, dim=1)

        pred = DICT[argmax.item()][2], out[0, argmax].item()

        before_cam = CAM_FUNCTION(before_img, MODEL, TARGET, id)

        after_image = delete_the_least_important_pixels(before_img, before_cam, PERCENTAGE)

        before_cam = CAM_FUNCTION(after_image, MODEL, TARGET, id)

        MODEL.eval()
        out = MODEL(after_image.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        aft = out[0, id].item()

        after.append(out[0, id].item())

        if VISUALIZE:
            print('pred', *pred)
            print('before', name, bef)
            print('after', name, aft)
            overlay_plot_torch(before_img, before_cam, HEATMAP)
            overlay_plot_torch(after_image, before_cam, HEATMAP)

before_arr = np.array(before)
after_arr = np.array(after)

f'AD: {average_drop(before_arr, after_arr):.3f}, RIS: {rate_of_increase_in_score(before_arr, after_arr):.3f}'

/home/jakub/Documents/Tasks/final/grad-cam/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jakub/Documents/Tasks/final/grad-cam/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 200/200 [11:11<00:00,  3.36s/it]


'AD: 0.025, RIS: 0.387'

In [3]:
import random
import os
from tqdm import tqdm


MODEL = resnet50(pretrained=True)
TARGET = MODEL.layer4[-1].conv3

before = []
after = []

PERCENTAGE = 0.05
CLASSES = 200
IMG_PER_CLASS = 5
IN_IMG = 224

VISUALIZE = False

CAM_FUNCTION = get_cam

HEATMAP = 0.5


for dir_name in tqdm(random.sample(os.listdir('imagenet-mini/train'), CLASSES)):
    for file_name in random.sample(os.listdir(f'imagenet-mini/train/{dir_name}'), IMG_PER_CLASS):

        before_img = get_image_torch(f'imagenet-mini/train/{dir_name}/{file_name}', IN_IMG, IN_IMG)
        id, hash, name = DICT[dir_name]

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        bef = out[0, id].item()

        before.append(out[0, id].item())

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)
        argmax = torch.argmax(out, dim=1)

        pred = DICT[argmax.item()][2], out[0, argmax].item()

        before_cam = CAM_FUNCTION(before_img, MODEL, TARGET, id)

        after_image = delete_the_least_important_pixels(before_img, before_cam, PERCENTAGE)

        before_cam = CAM_FUNCTION(after_image, MODEL, TARGET, id)

        MODEL.eval()
        out = MODEL(after_image.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        aft = out[0, id].item()

        after.append(out[0, id].item())

        if VISUALIZE:
            print('pred', *pred)
            print('before', name, bef)
            print('after', name, aft)
            overlay_plot_torch(before_img, before_cam, HEATMAP)
            overlay_plot_torch(after_image, before_cam, HEATMAP)

before_arr = np.array(before)
after_arr = np.array(after)

f'AD: {average_drop(before_arr, after_arr):.3f}, RIS: {rate_of_increase_in_score(before_arr, after_arr):.3f}'

100%|██████████| 200/200 [11:27<00:00,  3.44s/it]


'AD: 0.065, RIS: 0.293'

In [4]:
import random
import os
from tqdm import tqdm


MODEL = resnet50(pretrained=True)
TARGET = MODEL.layer4[-1].conv3

before = []
after = []

PERCENTAGE = 0.1
CLASSES = 200
IMG_PER_CLASS = 5
IN_IMG = 224

VISUALIZE = False

CAM_FUNCTION = get_cam

HEATMAP = 0.5


for dir_name in tqdm(random.sample(os.listdir('imagenet-mini/train'), CLASSES)):
    for file_name in random.sample(os.listdir(f'imagenet-mini/train/{dir_name}'), IMG_PER_CLASS):

        before_img = get_image_torch(f'imagenet-mini/train/{dir_name}/{file_name}', IN_IMG, IN_IMG)
        id, hash, name = DICT[dir_name]

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        bef = out[0, id].item()

        before.append(out[0, id].item())

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)
        argmax = torch.argmax(out, dim=1)

        pred = DICT[argmax.item()][2], out[0, argmax].item()

        before_cam = CAM_FUNCTION(before_img, MODEL, TARGET, id)

        after_image = delete_the_least_important_pixels(before_img, before_cam, PERCENTAGE)

        before_cam = CAM_FUNCTION(after_image, MODEL, TARGET, id)

        MODEL.eval()
        out = MODEL(after_image.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        aft = out[0, id].item()

        after.append(out[0, id].item())

        if VISUALIZE:
            print('pred', *pred)
            print('before', name, bef)
            print('after', name, aft)
            overlay_plot_torch(before_img, before_cam, HEATMAP)
            overlay_plot_torch(after_image, before_cam, HEATMAP)

before_arr = np.array(before)
after_arr = np.array(after)

f'AD: {average_drop(before_arr, after_arr):.3f}, RIS: {rate_of_increase_in_score(before_arr, after_arr):.3f}'

100%|██████████| 200/200 [11:24<00:00,  3.42s/it]


'AD: 0.109, RIS: 0.239'

In [5]:
import random
import os
from tqdm import tqdm


MODEL = resnet50(pretrained=True)
TARGET = MODEL.layer4[-1].conv3

before = []
after = []

PERCENTAGE = 0.25
CLASSES = 200
IMG_PER_CLASS = 5
IN_IMG = 224

VISUALIZE = False

CAM_FUNCTION = get_cam

HEATMAP = 0.5


for dir_name in tqdm(random.sample(os.listdir('imagenet-mini/train'), CLASSES)):
    for file_name in random.sample(os.listdir(f'imagenet-mini/train/{dir_name}'), IMG_PER_CLASS):

        before_img = get_image_torch(f'imagenet-mini/train/{dir_name}/{file_name}', IN_IMG, IN_IMG)
        id, hash, name = DICT[dir_name]

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        bef = out[0, id].item()

        before.append(out[0, id].item())

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)
        argmax = torch.argmax(out, dim=1)

        pred = DICT[argmax.item()][2], out[0, argmax].item()

        before_cam = CAM_FUNCTION(before_img, MODEL, TARGET, id)

        after_image = delete_the_least_important_pixels(before_img, before_cam, PERCENTAGE)

        before_cam = CAM_FUNCTION(after_image, MODEL, TARGET, id)

        MODEL.eval()
        out = MODEL(after_image.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        aft = out[0, id].item()

        after.append(out[0, id].item())

        if VISUALIZE:
            print('pred', *pred)
            print('before', name, bef)
            print('after', name, aft)
            overlay_plot_torch(before_img, before_cam, HEATMAP)
            overlay_plot_torch(after_image, before_cam, HEATMAP)

before_arr = np.array(before)
after_arr = np.array(after)

f'AD: {average_drop(before_arr, after_arr):.3f}, RIS: {rate_of_increase_in_score(before_arr, after_arr):.3f}'

100%|██████████| 200/200 [10:17<00:00,  3.09s/it]


'AD: 0.227, RIS: 0.130'

In [6]:
import random
import os
from tqdm import tqdm


MODEL = resnet50(pretrained=True)
TARGET = MODEL.layer4[-1].conv3

before = []
after = []

PERCENTAGE = 0.5
CLASSES = 200
IMG_PER_CLASS = 5
IN_IMG = 224

VISUALIZE = False

CAM_FUNCTION = get_cam

HEATMAP = 0.5


for dir_name in tqdm(random.sample(os.listdir('imagenet-mini/train'), CLASSES)):
    for file_name in random.sample(os.listdir(f'imagenet-mini/train/{dir_name}'), IMG_PER_CLASS):

        before_img = get_image_torch(f'imagenet-mini/train/{dir_name}/{file_name}', IN_IMG, IN_IMG)
        id, hash, name = DICT[dir_name]

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        bef = out[0, id].item()

        before.append(out[0, id].item())

        MODEL.eval()
        out = MODEL(before_img.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)
        argmax = torch.argmax(out, dim=1)

        pred = DICT[argmax.item()][2], out[0, argmax].item()

        before_cam = CAM_FUNCTION(before_img, MODEL, TARGET, id)

        after_image = delete_the_least_important_pixels(before_img, before_cam, PERCENTAGE)

        before_cam = CAM_FUNCTION(after_image, MODEL, TARGET, id)

        MODEL.eval()
        out = MODEL(after_image.unsqueeze(0))
        out = torch.nn.functional.softmax(out, dim=1)

        aft = out[0, id].item()

        after.append(out[0, id].item())

        if VISUALIZE:
            print('pred', *pred)
            print('before', name, bef)
            print('after', name, aft)
            overlay_plot_torch(before_img, before_cam, HEATMAP)
            overlay_plot_torch(after_image, before_cam, HEATMAP)

before_arr = np.array(before)
after_arr = np.array(after)

f'AD: {average_drop(before_arr, after_arr):.3f}, RIS: {rate_of_increase_in_score(before_arr, after_arr):.3f}'

100%|██████████| 200/200 [10:16<00:00,  3.08s/it]


'AD: 0.373, RIS: 0.056'